# ALGORITMO DE IA-4: REDE NEURAL CONVOLUCIONAL (CNN) 
# EXECUTADA SOBRE 1 TESLA K-80 GPU

ENUNCIADO:  
Implementar uma Rede Neural Convolucional (CNN) usando as bibliotecas TensorFlow e Keras

# CASO 03: DATASET CIFAR

CIFAR-10 é um conjunto de dados que consiste em 60'000 imagens coloridas de 32x32 distribuidas em 10 classes, 6'000 imagens por classe. São 50'000 imagens no conjunto de treinamento e 10'000 imagens para teste

In [2]:
#verificar conexão com GPU de GoogleColab
import tensorflow as tf
tf.test.gpu_device_name()

#verificar que GPU estou usando 
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8579116533636927463, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281989632
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14198208983889221277
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

1) Importar livrarías/formatear dataset

In [0]:
#importar livrarías
import pandas as pd
import numpy as np
import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from scipy.misc import toimage

In [4]:
#carregar conjunto de treinamento/teste
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 98s 1us/step


2) FASE DE PREPROCESSAMENTO

In [0]:
#Aplicar redimensionamento sobre os dados para obter uma matriz [#samples] x [width] x [weight] x [channels]

#conjunto de treinamento
x_train = X_train.reshape(X_train.shape[0], 32, 32, 3)

#conjunto de teste
x_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

In [0]:
#converter os tipos de dado para float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
#standarização dos dados: x_std = (x-ux)/std(x)
x_train = (x_train - np.mean(x_train))/np.std(x_train)
x_test = (x_test - np.mean(x_test))/np.std(x_test)

In [8]:
#informação sobre a matriz de caraterísticas:
dim_inputs = x_train.shape[2] * x_train.shape[3]
print('#Variáveis na matriz de caraterísticas: ', dim_inputs)
print('#Dim. Matriz de Caraterísticas - train: ', x_train.shape)
print('#Dim. Matriz de Caraterísticas - test: ', x_test.shape)

#Variáveis na matriz de caraterísticas:  96
#Dim. Matriz de Caraterísticas - train:  (50000, 32, 32, 3)
#Dim. Matriz de Caraterísticas - test:  (10000, 32, 32, 3)


In [9]:
#visualizar saídas/classes
print('Output train: ', np.unique(Y_train))

#transformar o vetor de classes em uma matriz binária de classes [0...9]
y_train = keras.utils.to_categorical(Y_train, len(np.unique(Y_train)))
y_test = keras.utils.to_categorical(Y_test, len(np.unique(Y_test)))

#informação sobre a matriz de saídas: 
dim_outputs = len(np.unique(Y_train))
print('Classes possíveis: ', dim_outputs)
print('Dim. Matriz de Saídas - train: ', y_train.shape)
print('Dim. Matriz de Saídas - test: ', y_test.shape)

Output train:  [0 1 2 3 4 5 6 7 8 9]
Classes possíveis:  10
Dim. Matriz de Saídas - train:  (50000, 10)
Dim. Matriz de Saídas - test:  (10000, 10)


3) FASE DE APRENDIZADO

In [0]:
#importar livrarías
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

Define-se a seguinte arquitetura de Rede Neural Convolucional para o CIFAR-10:

![texto alternativo](img/cifar_architect.png)

In [11]:
#criar arquitetura do modelo

#limite de plateau
weight_decay = 1e-4

#inicializar modelo
classifier = Sequential()

#1° Camada Convolucional
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding = 'same', input_shape = (32, 32, 3), 
                             activation = 'elu', kernel_regularizer = regularizers.l2(weight_decay)))
classifier.add(BatchNormalization())

#2° Camada Convolucional
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding = 'same', 
                             activation = 'elu', kernel_regularizer = regularizers.l2(weight_decay)))
classifier.add(BatchNormalization())

#1° Camada Max-Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#1° Camada DropOut
classifier.add(Dropout(0.2))

#####

#3° Camada Convolucional
classifier.add(Convolution2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same',
                             activation = 'elu', kernel_regularizer = regularizers.l2(weight_decay)))
classifier.add(BatchNormalization())

#4° Camada Convolucional
classifier.add(Convolution2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same',
                             activation = 'elu', kernel_regularizer = regularizers.l2(weight_decay)))
classifier.add(BatchNormalization())

#2° Camada Max-Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#2° Camada Dropout
classifier.add(Dropout(0.3))

#####

#5° Camada Convolucional
classifier.add(Convolution2D(filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same',
                             activation = 'elu', kernel_regularizer = regularizers.l2(weight_decay)))
classifier.add(BatchNormalization())

#6° Camada Convolucional
classifier.add(Convolution2D(filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same',
                             activation = 'elu', kernel_regularizer = regularizers.l2(weight_decay)))
classifier.add(BatchNormalization())

#3° Camada Max-Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#3° Camada Dropout
classifier.add(Dropout(0.4))

#Camada Flattening
classifier.add(Flatten())

#Camada Fully-connected
classifier.add(Dense(units = dim_outputs, activation = 'softmax'))

#Resumo do modelo
classifier.summary()

#Regularizacao com Data Argumentation
datagen = ImageDataGenerator(rotation_range = 15, 
                             width_shift_range = 0.1, 
                             height_shift_range = 0.1, 
                            horizontal_flip = True)

#algoritmo de otimizacao
opt_rms = keras.optimizers.rmsprop(lr = 0.001, decay = 1e-4)

#Compilar CNN
classifier.compile(optimizer = opt_rms, loss = 'categorical_crossentropy', metrics = ['accuracy'])

#fixar o modelo com o gerador

def lr_control(epoch):
    rate = 1e-3
    if epoch > 50:
        rate = 5e-4
    elif epoch>125:
        rate = 1e-4
    return rate

classifier.fit_generator(datagen.flow(x_train, y_train, batch_size = 64),
                        steps_per_epoch = x_train.shape[0] // 64 , epochs = 250,
                        validation_data = (x_test, y_test), verbose = 1,
                        callbacks = [LearningRateScheduler(lr_control)])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [0]:
#armazenar o modelo em disco
model_json = classifier.to_json()
with open('cnn_cifar.json', 'w') as json_file:
    json_file.write(model_json)
classifier.save_weights('cnn_cifar.h5')

4) FASE DE AVALIAÇÃO

In [13]:
#cálculo da acurácia no conjunto de treino
score_train = classifier.evaluate(x_train, y_train, batch_size = 64)
print('Acurácia-train: ', 100*score_train[1], '%')

50000/50000 [==============================] - 11s 212us/step
Acurácia-train:  94.838 %


In [14]:
#cálculo da acurácia no conjunto de teste
score_test = classifier.evaluate(x_test, y_test, batch_size = 64)
print('Acurácia-test: ', 100*score_test[1], '%')

10000/10000 [==============================] - 2s 226us/step
Acurácia-test:  88.94 %


5) CONCLUSÕES:  
Com o modelo de rede CNN proposta aplicada em 250 epochs executada sobre 1 Tesla K-80 GPU , os resultados são: 
- Acurácia no conjunto de treinamento: 94.84%
- Acurácia no conjunto de teste: 88.94%